In [2]:
import os
import numpy as np
from functions import *
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from mlp import MLP
import pickle

#boolean parameter to decide if i want to save the model weights and biases to a pickle file
save_model=True

# Hyperparameters
batch_size = 200  # Batch size for mini-batch gradient descent
learning_rate = 0.1  # Learning rate for weight updates
epochs = 20  # Number of epochs to train
input_size = 784  # Input size (28x28 flattened images)
output_size = 10  # Number of output neurons (10 classes for MNIST)
hidden_layer_sizes = [128, 128 ]  # Number of neurons in hidden

# array to store train loss per epoch
train_losses = []

train_loader, test_loader = load_mnist(batch_size)

# Initialize MLP model
mlp = MLP(learning_rate=learning_rate,input_size=input_size,output_size=output_size, hidden_layer_sizes=hidden_layer_sizes, activation_function='relu')

counter=0
# Training loop
for epoch in range(epochs):
    total_loss = 0  # Variable to store total loss for the epoch

    for batch_idx, (batch_images, batch_labels) in enumerate(train_loader):
        print(batch_idx)

        batch_images = batch_images.view(batch_size, -1)  # Flatten images to 1D vector (28x28 = 784)
        batch_labels = np.eye(output_size)[batch_labels]  # One-hot encode the labels

        # Perform forward pass: Get model predictions
        predictions = mlp.forward(batch_images.numpy())

        # Compute loss (Mean Squared Error)
        loss = cross_entropy_loss(batch_labels, predictions)
        total_loss += loss

        # Compute accuracy for the batch
        preds = np.argmax(predictions, axis=1)
        
        # Perform backpropagation
        gradients_w, gradients_b = mlp.back_propagation(batch_images.numpy(), batch_labels)

        # Update weights and biases
        mlp.update_weights(gradients_w, gradients_b)
    print(counter)
    # Compute average training loss and accuracy for the epoch
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Print loss after every epoch
    print(f"Epoch {epoch}, Loss: {avg_train_loss}")

# print(train_losses)

#save the model weights and biases to a pickle file
if(save_model): 
    #define the model parameters to be saved
    model_parameters = {
    "weights": mlp.weights,
    "biases": mlp.biases,
    "input_size": mlp.input_size,
    "hidden_layer_sizes": mlp.hidden_layer_sizes,
    "output_size": mlp.output_size,
    "learning_rate": mlp.learning_rate
    }

# Define the directory where the model will be saved
    save_dir = "models_self_implementation"
    
    # Create the directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Define the full path for the pickle file
    save_path = os.path.join(save_dir, "mlp_model.pkl")
    
    # Save the model parameters using pickle
    with open(save_path, "wb") as f:
        pickle.dump(model_parameters, f) 
    
    print(f"Model saved successfully in '{save_path}'")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
TRAIN=train_loader
TEST=test_loader

# Load Model
save_dir = "models_self_implementation"
model_filename = "mlp_model.pkl"
save_path = os.path.join(save_dir, model_filename)
if os.path.exists(save_path):
    with open(save_path, "rb") as f:
        model_parameters = pickle.load(f)

# Recreate the MLP with the same structure
mlp_imp = MLP(
    input_size=model_parameters["input_size"],
    hidden_layer_sizes=model_parameters["hidden_layer_sizes"],
    output_size=model_parameters["output_size"],
    learning_rate=model_parameters["learning_rate"]
)

# Assign saved weights and biases
mlp_imp.weights = model_parameters["weights"]
mlp_imp.biases = model_parameters["biases"]

# Evaluate model
all_accuracies = []
counter=0
for batch_images, batch_labels in TEST:
    counter+=1
    batch_images = batch_images.view(batch_images.shape[0], -1).numpy()
    batch_labels = batch_labels.numpy()
    acc = mlp_imp.evaluate(batch_images, batch_labels)
    all_accuracies.append(acc)
print(counter)
final_accuracy = np.mean(all_accuracies)
print("Test Accuracy:", round(final_accuracy*100,2), "%")

AttributeError: 'int' object has no attribute 'numpy'